## DataFrame&SQL

In [ ]:
!apt-get update

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
sc = spark.sparkContext

In [ ]:
import urllib
f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

In [ ]:
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file).cache()

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
from pyspark.sql import Row

csv_data = raw_data.map(lambda l: l.split(","))
row_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5])
    )
)

In [ ]:
interactions_df = sqlContext.createDataFrame(row_data)
interactions_df.registerTempTable("interactions")

In [ ]:
tcp_interactions = sqlContext.sql("""
    SELECT duration, dst_bytes FROM interactions WHERE protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0
""")
tcp_interactions.show()

+--------+---------+
|duration|dst_bytes|
+--------+---------+
|    5057|        0|
|    5059|        0|
|    5051|        0|
|    5056|        0|
|    5051|        0|
|    5039|        0|
|    5062|        0|
|    5041|        0|
|    5056|        0|
|    5064|        0|
|    5043|        0|
|    5061|        0|
|    5049|        0|
|    5061|        0|
|    5048|        0|
|    5047|        0|
|    5044|        0|
|    5063|        0|
|    5068|        0|
|    5062|        0|
+--------+---------+
only showing top 20 rows



In [ ]:
interactions_df.select('duration','dst_bytes').filter((interactions_df.protocol_type=="tcp") & (interactions_df.duration>1000) & (interactions_df.dst_bytes=='0')).show()

+--------+---------+
|duration|dst_bytes|
+--------+---------+
|    5057|        0|
|    5059|        0|
|    5051|        0|
|    5056|        0|
|    5051|        0|
|    5039|        0|
|    5062|        0|
|    5041|        0|
|    5056|        0|
|    5064|        0|
|    5043|        0|
|    5061|        0|
|    5049|        0|
|    5061|        0|
|    5048|        0|
|    5047|        0|
|    5044|        0|
|    5063|        0|
|    5068|        0|
|    5062|        0|
+--------+---------+
only showing top 20 rows



In [ ]:
tcp_interactions_out = tcp_interactions.rdd.map(lambda p: "Duration: {}, Dest. bytes: {}".format(p.duration, p.dst_bytes))
for ti_out in tcp_interactions_out.collect():
  print (ti_out)

Duration: 5057, Dest. bytes: 0
Duration: 5059, Dest. bytes: 0
Duration: 5051, Dest. bytes: 0
Duration: 5056, Dest. bytes: 0
Duration: 5051, Dest. bytes: 0
Duration: 5039, Dest. bytes: 0
Duration: 5062, Dest. bytes: 0
Duration: 5041, Dest. bytes: 0
Duration: 5056, Dest. bytes: 0
Duration: 5064, Dest. bytes: 0
Duration: 5043, Dest. bytes: 0
Duration: 5061, Dest. bytes: 0
Duration: 5049, Dest. bytes: 0
Duration: 5061, Dest. bytes: 0
Duration: 5048, Dest. bytes: 0
Duration: 5047, Dest. bytes: 0
Duration: 5044, Dest. bytes: 0
Duration: 5063, Dest. bytes: 0
Duration: 5068, Dest. bytes: 0
Duration: 5062, Dest. bytes: 0
Duration: 5046, Dest. bytes: 0
Duration: 5052, Dest. bytes: 0
Duration: 5044, Dest. bytes: 0
Duration: 5054, Dest. bytes: 0
Duration: 5039, Dest. bytes: 0
Duration: 5058, Dest. bytes: 0
Duration: 5051, Dest. bytes: 0
Duration: 5032, Dest. bytes: 0
Duration: 5063, Dest. bytes: 0
Duration: 5040, Dest. bytes: 0
Duration: 5051, Dest. bytes: 0
Duration: 5066, Dest. bytes: 0
Duration

In [ ]:
#tcp_interactions_df = sqlContext.createDataFrame(tcp_interactions)
tcp_interactions.registerTempTable("tcp_interactions")

In [ ]:
a=sqlContext.sql("""
    SELECT  CONCAT("Duration: ",duration) AS DURATION ,CONCAT(" DEST. bytes: ",dst_bytes) AS DST_BYTES 
    FROM  tcp_interactions
""")
a.show()

+--------------+---------------+
|      DURATION|      DST_BYTES|
+--------------+---------------+
|Duration: 5057| DEST. bytes: 0|
|Duration: 5059| DEST. bytes: 0|
|Duration: 5051| DEST. bytes: 0|
|Duration: 5056| DEST. bytes: 0|
|Duration: 5051| DEST. bytes: 0|
|Duration: 5039| DEST. bytes: 0|
|Duration: 5062| DEST. bytes: 0|
|Duration: 5041| DEST. bytes: 0|
|Duration: 5056| DEST. bytes: 0|
|Duration: 5064| DEST. bytes: 0|
|Duration: 5043| DEST. bytes: 0|
|Duration: 5061| DEST. bytes: 0|
|Duration: 5049| DEST. bytes: 0|
|Duration: 5061| DEST. bytes: 0|
|Duration: 5048| DEST. bytes: 0|
|Duration: 5047| DEST. bytes: 0|
|Duration: 5044| DEST. bytes: 0|
|Duration: 5063| DEST. bytes: 0|
|Duration: 5068| DEST. bytes: 0|
|Duration: 5062| DEST. bytes: 0|
+--------------+---------------+
only showing top 20 rows



In [ ]:
from time import time

t0 = time()
interactions_df.select("protocol_type").groupBy("protocol_type").count().show()
tt = time() - t0

print ("Query performed in {} seconds".format(round(tt,3)))

+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|190065|
|          udp| 20354|
|         icmp|283602|
+-------------+------+

Query performed in 8.456 seconds


In [ ]:
from time import time
t0 = time()
a=sqlContext.sql("""
    SELECT  protocol_type,COUNT(protocol_type) 
    FROM interactions 
    GROUP BY protocol_type
""")
a.show()
tt = time() - t0
print("Query performed in {} seconds".format(round(tt,3)))

+-------------+--------------------+
|protocol_type|count(protocol_type)|
+-------------+--------------------+
|          tcp|              190065|
|          udp|               20354|
|         icmp|              283602|
+-------------+--------------------+

Query performed in 7.672 seconds


dataframe operation :4

In [ ]:
t0 = time()
interactions_df.select("protocol_type", "duration", "dst_bytes").filter(interactions_df.duration>1000).filter(interactions_df.dst_bytes==0).groupBy("protocol_type").count().show()
tt = time() - t0

print ("Query performed in {} seconds".format(round(tt,3)))

+-------------+-----+
|protocol_type|count|
+-------------+-----+
|          tcp|  139|
+-------------+-----+

Query performed in 6.764 seconds


sql operation:4


In [ ]:
from time import time
t0 = time()
a=sqlContext.sql("""
    SELECT  protocol_type,COUNT(protocol_type) 
    FROM interactions 
    WHERE (duration>1000 AND dst_bytes==0)
    GROUP BY protocol_type
    
""")
a.show()
tt = time() - t0
print("Query performed in {} seconds".format(round(tt,3)))

+-------------+--------------------+
|protocol_type|count(protocol_type)|
+-------------+--------------------+
|          tcp|                 139|
+-------------+--------------------+

Query performed in 6.635 seconds


In [ ]:
def get_label_type(label):
    if label!="normal.":
        return "attack"
    else:
        return "normal"
    
row_labeled_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5]),
    label=get_label_type(p[41])
    )
)
interactions_labeled_df = sqlContext.createDataFrame(row_labeled_data)

In [ ]:
t0 = time()
interactions_labeled_df.select("label").groupBy("label").count().show()
tt = time() - t0

print ("Query performed in {} seconds".format(round(tt,3)))

+------+------+
| label| count|
+------+------+
|normal| 97278|
|attack|396743|
+------+------+

Query performed in 6.923 seconds


In [ ]:
interactions_labeled_df.registerTempTable("interactions_labeled")

In [ ]:
from time import time
t0 = time()
a=sqlContext.sql("""
    SELECT  label,COUNT(label) 
    FROM interactions_labeled 
    GROUP BY label
    
""")
a.show()
tt = time() - t0
print("Query performed in {} seconds".format(round(tt,3)))

+------+------------+
| label|count(label)|
+------+------------+
|normal|       97278|
|attack|      396743|
+------+------------+

Query performed in 6.807 seconds


In [ ]:
t0 = time()
interactions_labeled_df.select("label", "protocol_type").groupBy("label", "protocol_type").count().show()
tt = time() - t0

print ("Query performed in {} seconds".format(round(tt,3)))

+------+-------------+------+
| label|protocol_type| count|
+------+-------------+------+
|normal|          udp| 19177|
|normal|         icmp|  1288|
|normal|          tcp| 76813|
|attack|         icmp|282314|
|attack|          tcp|113252|
|attack|          udp|  1177|
+------+-------------+------+

Query performed in 7.16 seconds


In [ ]:
from time import time
t0 = time()
a=sqlContext.sql("""
    SELECT  label,protocol_type,COUNT(label) 
    FROM interactions_labeled 
    GROUP BY label,protocol_type
    
""")
a.show()
tt = time() - t0
print("Query performed in {} seconds".format(round(tt,3)))

+------+-------------+------------+
| label|protocol_type|count(label)|
+------+-------------+------------+
|normal|          udp|       19177|
|normal|         icmp|        1288|
|normal|          tcp|       76813|
|attack|         icmp|      282314|
|attack|          tcp|      113252|
|attack|          udp|        1177|
+------+-------------+------------+

Query performed in 7.056 seconds


In [ ]:
t0 = time()
interactions_labeled_df.select("label", "protocol_type", "dst_bytes").groupBy("label", "protocol_type", interactions_labeled_df.dst_bytes==0).count().show()
tt = time() - t0

print ("Query performed in {} seconds".format(round(tt,3)))

+------+-------------+---------------+------+
| label|protocol_type|(dst_bytes = 0)| count|
+------+-------------+---------------+------+
|normal|          udp|          false| 15583|
|attack|          udp|          false|    11|
|attack|          tcp|           true|110583|
|normal|          tcp|          false| 67500|
|attack|         icmp|           true|282314|
|attack|          tcp|          false|  2669|
|normal|          tcp|           true|  9313|
|normal|          udp|           true|  3594|
|normal|         icmp|           true|  1288|
|attack|          udp|           true|  1166|
+------+-------------+---------------+------+

Query performed in 7.171 seconds


In [ ]:
t0 = time()
a = sqlContext.sql("""
                           SELECT
                             label,protocol_type,
                             CASE 
                               WHEN interactions_labeled.dst_bytes>0 THEN 'false'
                               ELSE 'true'
                             END AS dst_typeequal0 ,
                             COUNT(*) as count
                           FROM interactions_labeled
                           GROUP BY label,protocol_type, dst_typeequal0
                           
                           """)
a.show()
tt = time() - t0

print ("Query performed in {} seconds".format(round(tt,3)))

+------+-------------+--------------+------+
| label|protocol_type|dst_typeequal0| count|
+------+-------------+--------------+------+
|attack|          udp|          true|  1166|
|attack|          udp|         false|    11|
|normal|          udp|          true|  3594|
|attack|          tcp|          true|110583|
|attack|          tcp|         false|  2669|
|attack|         icmp|          true|282314|
|normal|          udp|         false| 15583|
|normal|          tcp|          true|  9313|
|normal|         icmp|          true|  1288|
|normal|          tcp|         false| 67500|
+------+-------------+--------------+------+

Query performed in 7.187 seconds
